In [ ]:
def test_model(name, main_model):
    model = main_model
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print name, "test accuracy score:", accuracy_score(y_test, model.predict(x_test))
    
test_model("KNN", KNeighborsClassifier())
test_model("Logistic Regression", LogisticRegression())
test_model("Gaussian Naive Bayes", GaussianNB())
test_model("SVM Classifier", SVC())
test_model("Decision Tree", DecisionTreeClassifier())
test_model("Random Forest", RandomForestClassifier())


Look at feature importance

model = RandomForestClassifier()
model.fit(x_train, y_train)
importances = model.feature_importances_
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

plt.figure(figsize = (25,10))
plt.title("Feature importances")
plt.bar(range(len(indices)), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(len(indices)), X.columns[indices], rotation = 90)
plt.xlim([-1, len(indices)])
plt.show()

Create dataframe with confusion matrix metrics for each feature in model
cm_features = ['AVG_YRLY_ARR','RELAGE','TOT_PRARR', 'TMSRV', 'AGE_FIRST_OFF','SNTLN_YRS', 'SMPOFF5','J001PJP1']
model_list = []
for feature in cm_features:
    model = "y, X = dmatrices('RESP ~ " +feature+ "', data=df2, return_type='dataframe')"
    model_list.append(model)
pprint.pprint(model_list)

from sklearn.metrics import roc_auc_score

#roc curves
def curves(name, classifier):
    probas_ = classifier.fit(x_train, y_train).predict_proba(x_test)
    fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label = "%s area under curve = %0.2f" 
             % (name, roc_auc_score(y_test, probas_[:, 1])))
    plt.legend(loc = "best")
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title("ROC curves")
    
curves("KNN", KNeighborsClassifier())
curves("Logistic Regression", LogisticRegression())
curves("Gaussian Naive Bayes", GaussianNB())
curves("SVM Classifier", SVC(probability = True))
curves("Decision Tree", DecisionTreeClassifier())
curves("Random Forest", RandomForestClassifier())

#grid search svc:
"""y, X = dmatrices('RESP ~ ADTYP + np.log(A001CNT+1) + SEX + RACE  + ETHNIC +RELTYP + J001PJP1+ np.log(RELAGE+1) + np.log(TMSRV+1) + np.log(TOT_PRARR+1) + PCTSRV + np.log(AGE_FIRST_OFF) + SMPOFF5+ A001FM1 + C(TREATMENT) + SNTLN_YRS + np.log(AVG_YRLY_ARR + 1)', data=df2, return_type='dataframe')

y = np.ravel(y)

x_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=7)

std_scale = preprocessing.StandardScaler().fit(x_train)
x_train = std_scale.transform(x_train)
x_test = std_scale.transform(x_test)

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(x_train, y_train)

    print("Best parameters set found on development set:")
    print(clf.best_params_)
    print("Grid scores on development set:")
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))

    print("Detailed classification report:")
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    y_true, y_pred = y_test, clf.predict(x_test)
    print(classification_report(y_true, y_pred))"""